# Demo of Integrating MeMaS with ChatGPT

This is a simple demo demonstrating one way of integrating MeMaS with ChatGPT. Ultimately, the MeMaS APIs of `memorize` and `recall` were designed to be basic and intuitive; the developers should have a lot freedom in how they want to utilize MeMaS. 

Think of MeMaS as the memory capabilities in your brain. You spontaneously memorize conversations, textbooks, memes, etc. And when you see a fresh baked apple pie, you may remember how to make an apple pie, but you may also remember your grandma loves to bake apple pies. MeMaS retrieves related information, but usefullness is to be determined by the chatbot/agent.

In [1]:
import openai
from pprint import pprint
import time


# TODO: put your own api key here, DO NOT COMMIT IT!!!
openai.api_key = ""

## Chat Experience without MeMaS
Let's start with how chatting with ChatGPT normally would be like

In [2]:
# Note that this starts a new chat each time! This importantly distinguishes between short term conversation context and long term memory
def chat_with_chatgpt(prompt, model="gpt-3.5-turbo", max_tokens=1000):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        max_tokens=max_tokens,
        n=1,
        stop=None,
        temperature=0,
    )

    message = response.choices[0].message.content
    return message

In [3]:
message1 = "I have nut allergy. What are some weird ice cream flavors you would recommend?"

message2 = "My friend recommends me to try Ferrero Rocher, would you also recommend it?"

In [4]:
pprint(chat_with_chatgpt(message1))

("If you have a nut allergy, it's important to avoid ice cream flavors that "
 'contain nuts or have a risk of cross-contamination. However, there are still '
 'plenty of unique and delicious ice cream flavors you can enjoy! Here are '
 'some recommendations:\n'
 '\n'
 '1. Lavender Honey: A floral and sweet combination that offers a unique '
 'taste.\n'
 '2. Matcha Green Tea: A popular Japanese flavor with a slightly bitter and '
 'earthy taste.\n'
 '3. Rosewater Cardamom: A fragrant and exotic flavor that combines floral and '
 'spicy notes.\n'
 '4. Balsamic Strawberry: A tangy and sweet combination that adds a twist to '
 'traditional strawberry ice cream.\n'
 '5. Black Sesame: A rich and nutty flavor without actual nuts, commonly found '
 'in Asian-inspired ice creams.\n'
 '6. Earl Grey: A creamy and aromatic ice cream infused with the flavors of '
 'Earl Grey tea.\n'
 '7. Lemon Basil: A refreshing and zesty flavor that combines the brightness '
 'of lemon with the herbaceousness of

In [5]:
# Unsurprisingly, ChatGPT does not remember our message1
pprint(chat_with_chatgpt(message2))

("As an AI language model, I don't have personal preferences or tastes. "
 'However, Ferrero Rocher is a popular chocolate brand known for its unique '
 'combination of a whole roasted hazelnut, a creamy filling, a crisp wafer '
 'shell, and a chocolate coating. Many people enjoy its rich and indulgent '
 'taste. If you like chocolate and hazelnuts, it might be worth giving Ferrero '
 'Rocher a try to see if you enjoy it as well.')


## Chat Experience with MeMaS
Now let's try integrating with MeMaS

First, create the Control Plane client. This is the intended client for Admins and (Human) Developers to manage MeMaS.

Use this client for management operations like creating users, creating corpora, deleting, etc.

In [6]:
# Use the memas_sdk to configure MeMaS setup your "table"
import memas_sdk
from memas_sdk.apis.tags import cp_api

host_ip = "localhost"
host_port = 8010

# The control plane client is essentially an admin/developer client.
admin_client = cp_api.CpApi(memas_sdk.ApiClient(memas_sdk.Configuration(
    host = f"http://{host_ip}:{host_port}"
)))

Using the Control Plane client, create a user and a corpus to house our data. A **Corpus** is MeMaS's unit of data storage, somewhat akin to a dataset/collection/database table. 

In [7]:
USER_PATHNAME = "chatgpt_demo_user1"
CONVO_PATHNAME = f"{USER_PATHNAME}:conversation_demo"

admin_client.create_user({"namespace_pathname": USER_PATHNAME})
admin_client.create_corpus({"corpus_pathname": CONVO_PATHNAME, "corpus_type": "conversation"})

ApiResponseFor200(response=<urllib3.response.HTTPResponse object at 0x7f6026d31930>, body=DynamicSchema({'success': <DynamicSchema: True>}), headers=<memas_sdk.schemas.Unset object at 0x7f6026cdea70>)

Now create the Data Plane client. This is the intended client for Chatbots and Agents interfacing with MeMaS. 

At its core are 2 APIs:
- **Memorize**: store the memory into MeMaS 
- **Recall**: query MeMaS to return whatever is related to a given clue

*NOTE: While in this demo we are using the same Jupyter notebook, usually you should not need to import both the `memas_client` and the `memas_sdk` due to the different intended users*

In [8]:
import memas_client
from memas_client.apis.tags import dp_api

# The data plane client is the client you want your chatbot interfacing with, and something you can give full control over to AI agents
memory = dp_api.DpApi(memas_client.ApiClient(memas_client.Configuration(
    host = f"http://{host_ip}:{host_port}"
)))

Now let's integrate MeMaS with ChatGPT through prompts.

In [9]:
wrapper_prompt = \
    "Do not treat the section wrapped in `$$$` as part of the prompt, instead treat it as past conversations. "\
    "$$${background_info}$$$\n"\
    "Now, putting this context aside, respond to this user prompt: \n" \
    "{actual_prompt}"


# Integrate by adding MeMaS as a chatbot wrapper, in between the user and the chatbot
def chat_with_chatgpt_memas(prompt):
    recall_response = memory.recall({"clue": prompt, "namespace_pathname": USER_PATHNAME})
    background_info = [o["document"] for o in recall_response.body][0:3]
    # print(background_info)
    
    chat_response = chat_with_chatgpt(wrapper_prompt.format(background_info=background_info, actual_prompt=prompt))
    
    # Memorize this conversation as well.
    wrapped_convo = f"USER: {prompt}\nCHATGPT: {chat_response}"
    memory.memorize({"document": wrapped_convo, "corpus_pathname": CONVO_PATHNAME, "citation": {}})
    
    return chat_response

In [10]:
pprint(chat_with_chatgpt_memas(message1))

("If you have a nut allergy, it's important to be cautious when trying out new "
 'ice cream flavors. While I can suggest some unique and interesting options, '
 'please make sure to check the ingredients and consult with the staff to '
 'ensure they are safe for you. Here are a few unusual ice cream flavors you '
 'might find intriguing:\n'
 '\n'
 '1. Lavender Honey: This floral-infused ice cream offers a delicate and '
 'aromatic flavor that can be quite refreshing.\n'
 '2. Balsamic Strawberry: A combination of tangy balsamic vinegar and sweet '
 'strawberries creates a unique and unexpected taste.\n'
 "3. Olive Oil: Don't let the name fool you! Olive oil ice cream can be "
 'surprisingly creamy and smooth, with a subtle fruity undertone.\n'
 '4. Rosemary Lemon: The herbaceous notes of rosemary combined with zesty '
 'lemon create a refreshing and slightly savory ice cream flavor.\n'
 "5. Black Sesame: While it's important to ensure there are no "
 'cross-contamination risks, black s

In [11]:
# sleep 1 second to make sure the memory is stored before we query
time.sleep(1)

# Now notice how when responding to the second message, it now remembers information from the first
pprint(chat_with_chatgpt_memas(message2))

("As an AI language model, I can't have personal opinions or taste "
 'preferences. However, I can provide you with information. Ferrero Rocher is '
 'a popular chocolate candy that typically contains hazelnuts. Since you '
 "mentioned having a nut allergy, it's important to be cautious. It's best to "
 'check the ingredients of Ferrero Rocher or consult with the manufacturer to '
 'ensure it is safe for you to consume.')


While this is an extremely basic demo, there is much more to come! 

One of the core features when designing MeMaS was to store knowledge. Please look forward to seeing how integrating with knowledge corpuses can reduce hallucination and reduce the need of fine tuning.